In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [70]:
# Import Libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 as cv
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
# Gọi các thư viện cần thiết 
import numpy as np
import pandas as pd # Xu lý bảng
import seaborn as sns # Vẽ biểu đồ thị của dữ liệu
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler # Xử lý chuẩn hóa dữ liệu
from sklearn.model_selection import train_test_split # Chia dữ liệu ra làm 2 phần
from keras.layers import Dense, Activation, Dropout, BatchNormalization, LSTM    # LSTM  biên dạng ANN, BatchNormalization: cho nhỏ lại
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical # Sử dung để làm nổi đối tượng cần phân loại
from keras import callbacks 
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score # Để đo lường

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation, Dropout, BatchNormalization, LSTM
from keras.models import Sequential 
from tensorflow.keras.utils import to_categorical
from keras import callbacks
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
import keras
from keras.models import Sequential
from keras.layers import Dense # fully connected
from keras.datasets import boston_housing
from tensorflow.keras.optimizers import RMSprop # toi uu
from keras.callbacks import EarlyStopping # dung lai ngay lap tuc
from sklearn.preprocessing import scale # xu li du lieu
from sklearn.preprocessing import StandardScaler # xu li du lieu

In [25]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory="../input/taitrain/taitrain/",
                                                 shuffle=True,
                                                 target_size=(32,32), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory="../input/taitrain/taitrain/",
                                                 shuffle=True,
                                                 target_size=(32,32), 
                                                 subset="validation",
                                                 class_mode='categorical')

In [26]:
train_dataset.class_indices

In [27]:
train_dataset.classes

In [37]:
import glob
notai = glob.glob('../input/taitrain/taitrain/NoTai/*.*')
tai = glob.glob('../input/taitrain/taitrain/Tai/*.*')


data = []
labels = []

for i in tai:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (32,32))
    image=np.array(image)
    data.append(image)
    labels.append(0)
for i in notai:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (32,32))
    image=np.array(image)
    data.append(image)
    labels.append(0)

data = np.array(data)
labels = np.array(labels)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.2,
                                                random_state=1)

In [38]:
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [42]:
# Preprocessing
x_train = X_train.reshape(94,3072)
x_test = X_test.reshape(24,3072)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /=255
x_test /=255
y_train = np_utils.to_categorical(Y_train,2)
y_test = np_utils.to_categorical(Y_test,2)

# Create model
model = Sequential()
model.add(Dense(512,activation = 'relu',input_shape=(3072,)))
model.add(Dropout(0.25))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(500,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(2))
#model.add(Dense(1))
model.summary()

In [43]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics = ['accuracy']) # bien dich


In [44]:
# Train model
history = model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=1,validation_data=(x_test,y_test),callbacks=[EarlyStopping(monitor='val_loss',patience=20)])

In [45]:
# Results
score = model.evaluate(x_test,y_test, verbose=0)
print("Loss = ", score[0])
print("accuracy = ", score[1])

In [46]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train','Validation'])
plt.show()


In [49]:
from tensorflow.keras.models import load_model
model.save('Final1.h5')
model_ANN = load_model('Final1.h5')

In [66]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
filename = "../input/taisstest/tai2.png"

predict = ["It's me","Not me"]
predict = np.array(predict)

img = load_img(filename,target_size=(32,32))
img = img_to_array(img)
img = img.reshape(1,32*32*3)
img = img.astype('float32')
img = img/255

result = np.argmax(model_ANN.predict(img),axis=-1)
predict[result]

In [71]:
from tensorflow.keras.utils import load_img, img_to_array
img = image.load_img("../input/taisstest/tai2.png")
plt.imshow(img)